# 解析の手順

LAMMPSの原子座標から、ネットワークを生成します。

In [ ]:
./lammps2ngph.py < test.lammpstrj > test.ngph

ネットワークから環を抽出します。

In [ ]:
./cycles.py < test.ngph > test.cycles.json

グラフのデータベース(graph.sqlite3)を準備します。これは、同じ形のグラフには同じIDが振られるようにするためです。以下のコマンドは、graph.sqlite3が存在しない時に一回だけ実行します。

以下のグラフに優先的にIDが設定されます。
* id 1: 氷Icのフラグメント $6^4$
* id 2: 氷Ihの六角柱フラグメント $6^5$
* id 3: Large barrelan, 五角の樽 $6^7$
* id 4: 正十二面体 $5^{12}$


In [ ]:
./prepare_db.py

環を組みあわせて多面体を作ります。(非常に時間かかります。近いうちにアルゴリズムを改善します)

この過程で、見付かった多面体がisomorphicかどうかをチェックし、新しい多面体であればどんどんデータベースに追加します。これにより、すべてのグラフの種類に固有のIDが与えられます。

In [ ]:
./polyhed.py < test.cycles.json > test.poly.ngph

このファイルの中にも、各多面体のIDが記録されているので、それを抽出し、数え、ソートすると、どの多面体か最も頻繁に生じているかがわかります。

In [ ]:
fgrep id test.poly.ngph | sort | uniq -c | sort -n

: 

```
   2 #id 4   正12面体

 ...

 185 #id 3   large barrelan (五角の樽。四面体クラスターの辺)
 230 #id 24
 272 #id 95
 414 #id 23  *
 488 #id 15  *
 572 #id 6   *
 635 #id 14  *
1039 #id 21  Fragment B
3147 #id 2   ice Ihの5面体 (六角柱)
4825 #id 10  small barrelan (ice Ihの小さいほうの多面体) Fragment A
9514 #id 1   ice Icの4面体
```
第一カラムが出現頻度、第3カラムがID。*がついているものは、どれも対称性の良い構造のそばで5+7欠陥が生じて乱れたもののように見える。



原子位置と多面体のリストを読みこみ、yaplot化します。
* graph.sqlite3に記録されたIDを照会して、同じグラフには同じ色がつきます。
* ID1のグラフがレイヤ1に、2のグラフがレイヤ2に表示されます。

In [ ]:
./lammps2yap2.py test.poly.ngph < test.lammpstrj > test.poly.yap


: 